<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/tivdak_gpt_3_5_turbo_gg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs and Setup

In [ ]:
!mkdir -p DATA/TIVDAK;curl https://docs.seagen.com/Tivdak_Full_Ltr_Master.pdf -o DATA/TIVDAK/tivdak.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  548k  100  548k    0     0   544k      0  0:00:01  0:00:01 --:--:--  544k


In [ ]:
%pip install -qq llama-index openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.9/607.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.0/255.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 37.6 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=2ff9e50f10d87a830a2fe3816e77aced37b703bb213728297f1c8f900ff531ba
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzloc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import os
import llama_index
from llama_index import ListIndex, VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext, PromptHelper, GPTVectorStoreIndex
from llama_index.response.notebook_utils import display_response
from llama_index import download_loader
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import logging, sys, os, json
from pathlib import Path
import openai
from typing import List, Tuple, Dict, Union

import streamlit as st

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
#all the things that are in lamaindex:
#dir(llama_index)

## Set up Open AI API key and data path

In [ ]:
test: int = 3
output: str = json.dumps({"test":test});output

'{"test": 3}'

In [ ]:
cwd: Path = Path.cwd()
data_dir: Path = cwd / "DATA"
tivdak_dir: Path  = data_dir / "TIVDAK"
documents = SimpleDirectoryReader(tivdak_dir).load_data()

In [ ]:
def set_open_ai_key(env_path: Union[Path, str] = None) -> Tuple[bool, str]:
  import json, os
  from pathlib import Path

  openai.api_key = None #clear previous key if exists
  env_path: Path = Path(env_path).absolute() # in case env_path is passed as a str
  if not env_path.is_file(): err: str = f"File:{str(env_path)} does not exist."
  else:
    try:
      with open(env_path, "r") as f:
          env_vars = json.load(f)
      os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
      openai.api_key = os.environ["OPENAI_API_KEY"]
      openai.Model.list() #test a random command on the openai API
      err = None
    except Exception as e:
      err = json.dumps({"error":e})

  logger.info(err)
  return (True, "ok") if not err else (False, err)

In [ ]:
#cwd: Path = Path.cwd()
env_path: Path = "/content/drive/MyDrive/Colab Notebooks/openai.env";
print(set_open_ai_key(env_path)) # Tuple[bool, str]

(True, 'ok')


## Call to OpenAI API (GPT 3.5)

In [ ]:
llm: llama_index.llms.openai.OpenAI = OpenAI(temperature=0,
             model="gpt-3.5-turbo",
             max_tokens=400,
             max_retries = 5)

service_context: llama_index.indices.service_context.ServiceContext = ServiceContext.from_defaults(llm=llm)

In [ ]:
davinci_index: llama_index.indices.vector_store.base.VectorStoreIndex = VectorStoreIndex.from_documents(documents, service_context=service_context)

## Chat with the Tivdak Docs

### basic Q&A applications

In [ ]:
query_engine: llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine = davinci_index.as_query_engine(response_mode="tree_summarize")
q: str = "what is tivdak used for?"
response: llama_index.response.schema.Response = query_engine.query(q)

In [ ]:
display_response(response)

**`Final Response:`** TIVDAK is used to treat adults with cervical cancer that has returned or has spread to other parts of the body, and who have received chemotherapy that did not work or is no longer working.

In [ ]:
q = "what is tisotumab vedotin used for?"
display_response(query_engine.query(q))

**`Final Response:`** Tisotumab vedotin is used for the treatment of adult patients with recurrent or metastatic cervical cancer with disease progression on or after chemotherapy.

In [ ]:
q = "what is tisotumab used for?"
display_response(query_engine.query(q))

**`Final Response:`** Tisotumab is used for the treatment of adult patients with recurrent or metastatic cervical cancer with disease progression on or after chemotherapy.

In [ ]:
q = "why do patients use cold packs when using tivdak?"
display_response(query_engine.query(q))

**`Final Response:`** Patients use cold packs when using TIVDAK to reduce the risk of eye problems.

In [ ]:
q = "what are the most common adverse reactions of tivdak"
display_response(query_engine.query(q))

**`Final Response:`** The most common adverse reactions of TIVDAK are fatigue, nausea, diarrhea, abdominal pain, peripheral neuropathy, alopecia, rash, epistaxis, hemorrhage, conjunctival adverse reactions, dry eye, and corneal adverse reactions.

In [ ]:
q = "what are the black box warnings of tivdak"
display_response(query_engine.query(q))

**`Final Response:`** The black box warnings of TIVDAK are not provided in the given context information.

In [ ]:
q = "what are the boxed warnings of tivdak"
display_response(query_engine.query(q))

**`Final Response:`** The boxed warnings of TIVDAK are not provided in the given context information.

In [ ]:
q = "what are the most common drug interactions of tivdak"
display_response(query_engine.query(q))

In [ ]:
q = "how is tivdak administered"
display_response(query_engine.query(q))

**`Final Response:`** TIVDAK is administered as an intravenous infusion over 30 minutes every 3 weeks until disease progression or unacceptable toxicity.

In [ ]:
q = "what are the Pharmacokinetics of tivdak"
display_response(query_engine.query(q))


**`Final Response:`** The pharmacokinetics of TIVDAK are not known.

In [ ]:
q = "what are the Pharmacokinetics of Tisotumab"
display_response(query_engine.query(q))


**`Final Response:`** The pharmacokinetics of Tisotumab are not known. It is not known how tisotumab is metabolized or excreted from the body.

In [ ]:
q = "what are the the exposure parameters of Tisotumab"
display_response(query_engine.query(q))


**`Final Response:`** The exposure parameters of Tisotumab Vedotin-tftv are summarized in Table 6. The mean maximum concentration (Cmax) of Tisotumab Vedotin-tftv is 40.8 μg/mL, and the mean area under the concentration-time curve (AUC) is 57.5 day*μg/mL. The mean Cmax of unconjugated MMAE (the cytotoxic component of Tisotumab Vedotin-tftv) is 5.91 ng/mL, and the mean AUC is 50 day*ng/mL.

In [ ]:
q = "what does the elimination of Tisotumab look like"
display_response(query_engine.query(q))

**`Final Response:`** The elimination of Tisotumab is characterized by a median terminal half-life of 4.04 days, a linear clearance of 1.54 L/day, and excretion of 17% of the total MMAE administered recovered in feces and 6% in urine over a 1-week period. It is thought to be metabolized in the liver and excreted in the urine and feces.

In [ ]:
q = "was the efficacy of TIVDAK ever evaluated? please cite any relevant clinical studies  "
display_response(query_engine.query(q))


**`Final Response:`** Yes, the efficacy of TIVDAK was evaluated. The clinical study mentioned in the context information is "innovaTV 204." This study evaluated the efficacy of TIVDAK in patients with cervical cancer. The efficacy results, including the confirmed overall response rate (ORR), complete response rate, partial response rate, and duration of response, are provided in Table 7 on page 18 of the document.

### Summmarize sections of report

In [ ]:
q = "Summarize the possible adverse affects of TIVDAK"
display_response(query_engine.query(q))

**`Final Response:`** The possible adverse effects of TIVDAK include periorbital adverse reactions, musculoskeletal and connective tissue disorders such as myalgia and arthralgia, pain in extremity, decreased appetite, urinary tract infection, weight decreased, fatigue, nausea, diarrhea, abdominal pain, peripheral neuropathy, rash, hemorrhage, conjunctival adverse reactions, dry eye, corneal adverse reactions, and venous thrombosis, pulmonary embolism, and pneumonitis in less than 10% of patients.

In [ ]:
q = "Summarize section 5 of the TIVDAK documentation"
display_response(query_engine.query(q))

**`Final Response:`** Section 5 of the TIVDAK documentation discusses the warnings and precautions associated with the use of the medication. It highlights three main concerns: peripheral neuropathy, hemorrhage, and pneumonitis. The section advises monitoring patients for new or worsening peripheral neuropathy and adjusting the dose or discontinuing TIVDAK accordingly. It also emphasizes the need to monitor for signs and symptoms of hemorrhage and to take appropriate action based on severity. Additionally, it warns about the potential occurrence of severe, life-threatening, or fatal pneumonitis and provides guidelines for managing it. The section also mentions the potential embryo-fetal toxicity of TIVDAK and advises patients to use effective contraception.

## Streamlit app interface

In [ ]:
doc_path = './data/'
index_file = 'index.json'

In [ ]:
if 'response' not in st.session_state:
    st.session_state.response = ''

def send_click():
    st.session_state.response  = index.query(st.session_state.prompt)

index = None
st.title("Tivdak Drug Info Chatbot")

sidebar_placeholder = st.sidebar.container()
uploaded_file = st.file_uploader("Choose a file")

if uploaded_file is not None:

    doc_files = os.listdir(doc_path)
    for doc_file in doc_files:
        os.remove(doc_path + doc_file)

    bytes_data = uploaded_file.read()
    with open(f"{doc_path}{uploaded_file.name}", 'wb') as f:
        f.write(bytes_data)

    SimpleDirectoryReader = download_loader("SimpleDirectoryReader")

    loader = SimpleDirectoryReader(doc_path, recursive=True, exclude_hidden=True)
    documents = loader.load_data()
    sidebar_placeholder.header('Current Processing Document:')
    sidebar_placeholder.subheader(uploaded_file.name)
    sidebar_placeholder.write(documents[0].get_text()[:10000]+'...')

    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-3.5-turbo"))

    max_input_size = 4096
    num_output = 256
    max_chunk_overlap = 20
    prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

    index = VectorStoreIndex.from_documents(
        documents, service_context=service_context
    )

    index.save_to_disk(index_file)

elif os.path.exists(index_file):
    index = VectorStoreIndex.load_from_disk(index_file)

    SimpleDirectoryReader = download_loader("SimpleDirectoryReader")
    loader = SimpleDirectoryReader(doc_path, recursive=True, exclude_hidden=True)
    documents = loader.load_data()
    doc_filename = os.listdir(doc_path)[0]
    sidebar_placeholder.header('Current Processing Document:')
    sidebar_placeholder.subheader(doc_filename)
    sidebar_placeholder.write(documents[0].get_text()[:10000]+'...')

if index != None:
    st.text_input("Ask something: ", key='prompt')
    st.button("Send", on_click=send_click)
    if st.session_state.response:
        st.subheader("Response: ")
        st.success(st.session_state.response, icon= "🤖")

2023-07-20 15:30:08.696 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2023-07-20 15:30:09.227 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
st run /content/drive/MyDrive/Colab Notebooks/streamlit_app/demo.py

SyntaxError: ignored

In [ ]:
streamlit run my_script.py